# Importando as Bibliotecas

In [1]:
# import pandas
import pandas as pd
import seaborn as sn

In [2]:
import spacy

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [4]:
import re

# Conhecendo os dados

In [5]:
df = pd.read_csv(filepath_or_buffer="data/Dataset_5971.csv", sep=",", encoding="latin-1")

In [6]:
display(df)

,LABEL,TEXT,URL,EMAIL,PHONE
0,ham,Your opinion about me? 1. Over 2. Jada 3. Kusr...,No,No,No
1,ham,What's up? Do you want me to come online? If y...,No,No,No
2,ham,So u workin overtime nigpun?,No,No,No
3,ham,"Also sir, i sent you an email about how to log...",No,No,No
4,Smishing,Please Stay At Home. To encourage the notion o...,No,No,No
...,...,...,...,...,...
5966,ham,:( but your not here....,No,No,No
5967,ham,Becoz its &lt;#&gt; jan whn al the post ofic...,No,No,No
5968,ham,Its a valentine game. . . send dis msg to all ...,No,No,No
5969,ham,We r outside already.,No,No,No


In [7]:
df.dtypes

LABEL    object
TEXT     object
URL      object
EMAIL    object
PHONE    object
dtype: object

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5971 entries, 0 to 5970
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   LABEL   5971 non-null   object
 1   TEXT    5971 non-null   object
 2   URL     5971 non-null   object
 3   EMAIL   5971 non-null   object
 4   PHONE   5971 non-null   object
dtypes: object(5)
memory usage: 233.4+ KB


In [9]:
df.describe()

,LABEL,TEXT,URL,EMAIL,PHONE
count,5971,5971,5971,5971,5971
unique,5,5949,2,2,2
top,ham,The greatest test of courage on earth is to be...,No,No,No
freq,4844,2,5764,5952,5127


In [10]:
df["LABEL"].unique()

array(['ham', 'Smishing', 'spam', 'Spam', 'smishing'], dtype=object)

In [11]:
df["URL"].unique()

array(['No', 'yes'], dtype=object)

In [12]:
df["EMAIL"].unique()

array(['No', 'yes'], dtype=object)

In [13]:
df["PHONE"].unique()

array(['No', 'yes'], dtype=object)

In [14]:
len([x for x in df.duplicated() if x == True])

17

In [15]:
i = 0
index_dup = []
for x in df.duplicated():
    if x == True:
        index_dup.append(i)
    i += 1

In [16]:
df.iloc[index_dup]

,LABEL,TEXT,URL,EMAIL,PHONE
1352,Smishing,UR GOING 2 BAHAMAS! CallFREEFONE 08081560665 a...,No,No,yes
1804,Smishing,Someone U know has asked our dating service 2 ...,No,No,yes
2963,ham,He remains a bro amongst bros,No,No,No
3692,ham,Reverse is cheating. That is not mathematics.,No,No,No
3874,ham,The sign of maturity is not when we start sayi...,No,No,No
3908,Smishing,U have a secret admirer. REVEAL who thinks U R...,No,No,yes
3909,Smishing,Congrats! 2 mobile 3G Videophones R yours. cal...,No,No,yes
4142,ham,The greatest test of courage on earth is to be...,No,No,No
4731,ham,"Sorry,in meeting I'll call later",No,No,No
4927,Smishing,URGENT!! Your 4* Costa Del Sol Holiday or Â£50...,No,No,yes


In [17]:
df[df["TEXT"] == "Sorry,in meeting I'll call later"]

,LABEL,TEXT,URL,EMAIL,PHONE
417,ham,"Sorry,in meeting I'll call later",No,No,No
4731,ham,"Sorry,in meeting I'll call later",No,No,No


In [18]:
df[df["TEXT"] == "Arun can u transfr me d amt"]

,LABEL,TEXT,URL,EMAIL,PHONE
415,ham,Arun can u transfr me d amt,No,No,No
5343,ham,Arun can u transfr me d amt,No,No,No


# Transformando os dados

In [19]:
df_transformado = df.copy()

In [20]:
df_transformado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5971 entries, 0 to 5970
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   LABEL   5971 non-null   object
 1   TEXT    5971 non-null   object
 2   URL     5971 non-null   object
 3   EMAIL   5971 non-null   object
 4   PHONE   5971 non-null   object
dtypes: object(5)
memory usage: 233.4+ KB


In [21]:
df_transformado["LABEL"].unique()

array(['ham', 'Smishing', 'spam', 'Spam', 'smishing'], dtype=object)

In [22]:
df_transformado["LABEL"].replace("Spam", "spam", inplace=True)
df_transformado["LABEL"].replace("Smishing", "smishing", inplace=True)
df_transformado["LABEL"].replace(["ham", "spam", "smishing"], [-1, 0, 1], inplace=True)

In [23]:
df_transformado["URL"].replace(["No", "yes"], [0, 1], inplace=True)
df_transformado["EMAIL"].replace(["No", "yes"], [0, 1], inplace=True)
df_transformado["PHONE"].replace(["No", "yes"], [0, 1], inplace=True)

In [24]:
display(df_transformado["URL"].unique())
display(df_transformado["EMAIL"].unique())
display(df_transformado["PHONE"].unique())

array([0, 1])

array([0, 1])

array([0, 1])

In [25]:
df_transformado.drop(["URL", "EMAIL", "PHONE"], axis=1, inplace=True)

In [26]:
len([x for x in df_transformado.duplicated() if x == True]) ## Ele aumenta por causa do tratamento que se fez para as palvras minusculas

20

In [27]:
df_transformado.drop_duplicates(inplace=True)

In [28]:
display(len(df_transformado))
display(len(df))

display(len(df) - len(df_transformado))

5951

5971

20

In [29]:
df_transformado.reset_index(drop=True, inplace=True)

In [30]:
display(df.head())
display(df_transformado.head())

,LABEL,TEXT,URL,EMAIL,PHONE
0,ham,Your opinion about me? 1. Over 2. Jada 3. Kusr...,No,No,No
1,ham,What's up? Do you want me to come online? If y...,No,No,No
2,ham,So u workin overtime nigpun?,No,No,No
3,ham,"Also sir, i sent you an email about how to log...",No,No,No
4,Smishing,Please Stay At Home. To encourage the notion o...,No,No,No


,LABEL,TEXT
0,-1,Your opinion about me? 1. Over 2. Jada 3. Kusr...
1,-1,What's up? Do you want me to come online? If y...
2,-1,So u workin overtime nigpun?
3,-1,"Also sir, i sent you an email about how to log..."
4,1,Please Stay At Home. To encourage the notion o...


# Analisando os dados

In [31]:
display(len(df_transformado[df_transformado["LABEL"] == -1]))
display(len(df_transformado[df_transformado["LABEL"] == 0]))
display(len(df_transformado[df_transformado["LABEL"] == 1]))

4834

489

628

# NLP

#### 1500 para treino e o resto para validação

In [32]:
!python -m spacy download en_core_web_trf

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 460.3 MB 36 kB/s                               | 5.3 MB 19.3 MB/s eta 0:00:24                             | 8.7 MB 19.3 MB/s eta 0:00:24         | 14.7 MB 19.3 MB/s eta 0:00:24                             | 19.0 MB 19.3 MB/s eta 0:00:23    |█▌                              | 21.8 MB 19.3 MB/s eta 0:00:23     | 31.4 MB 19.3 MB/s eta 0:00:234.7 MB 19.3 MB/s eta 0:00:23                     | 44.6 MB 97.7 MB/s eta 0:00:05:00:051.2 MB 97.7 MB/s eta 0:00:05:00:05��██▎                          | 75.1 MB 1.5 MB/s eta 0:04:22 :20  MB/s eta 0:04:18  | 91.7 MB 1.5 MB/s eta 0:04:11                  | 94.2 MB 1.5 MB/s eta 0:04:09 B/s eta 0:04:06 50:055               | 122.6 MB 71.9 MB/s eta 0:00:05               | 125.1 MB 71.9 MB/s eta 0:00:05                    | 128.1 MB 71.9 MB/s eta 0:00:05050:05B 130.5 MB/s eta 0:00:03��███▉                     | 155.6 MB 130.5 MB/s eta 0:00:03

In [33]:
nlp = spacy.load("en_core_web_trf")

In [34]:
df_teste = df_transformado.iloc[:1500].copy()
df_validacao = df_transformado.iloc[1500:].copy()

In [35]:
display(df_teste)
display(df_validacao)

,LABEL,TEXT
0,-1,Your opinion about me? 1. Over 2. Jada 3. Kusr...
1,-1,What's up? Do you want me to come online? If y...
2,-1,So u workin overtime nigpun?
3,-1,"Also sir, i sent you an email about how to log..."
4,1,Please Stay At Home. To encourage the notion o...
...,...,...
1495,0,EASTENDERS TV Quiz. What FLOWER does DOT compa...
1496,-1,"Ya, i'm referin to mei's ex wat... No ah, wait..."
1497,-1,"Bloody hell, cant believe you forgot my surnam..."
1498,-1,Don no da:)whats you plan? Let mw knowï¿½


,LABEL,TEXT
1500,-1,I gotta collect da car at 6 lei.
1501,-1,Hey... Thk we juz go accordin to wat we discus...
1502,-1,Ya it came a while ago
1503,-1,Appt is at &lt;TIME&gt; am. Not my fault u don...
1504,-1,For many things its an antibiotic and it can b...
...,...,...
5946,-1,I'm in class. Did you get my text.
5947,-1,:( but your not here....
5948,-1,Becoz its &lt;#&gt; jan whn al the post ofic...
5949,-1,Its a valentine game. . . send dis msg to all ...


In [36]:
def remover_espacos(text):
    frases_sem_espaco_frente = " ".join(text.split())
    return frases_sem_espaco_frente

In [37]:
def remover_caractere_especial(text):
    texto_sem_cs = re.sub("ï¿½", "", text)
    return texto_sem_cs

In [38]:
def remover_pontos(text):
    doc = nlp(text)
    frases_sem_ponto = " ".join([token.text for token in doc if not token.is_punct])
    return frases_sem_ponto

In [39]:
def lemmatizar(text):
    doc = nlp(text)
    frases_lematizadas = " ".join([token.lemma_ for token in doc])
    return frases_lematizadas

In [40]:
def pre_processamento(text):
    texto_processado = text
    print(texto_processado)
    texto_processado = remover_caractere_especial(texto_processado)
    texto_processado = remover_espacos(texto_processado)  
    texto_processado = remover_pontos(texto_processado)
    texto_processado = lemmatizar(texto_processado)
    print(texto_processado)
    return texto_processado

In [41]:
df_teste["PROCESSED_TEXT"] = df_teste["TEXT"].apply(pre_processamento)

Your opinion about me? 1. Over 2. Jada 3. Kusruthi 4. Lovable 5. Silent 6. Spl character 7. Not matured 8. Stylish 9. Simple Pls reply..
your opinion about I 1 over 2 Jada 3 Kusruthi 4 Lovable 5 Silent 6 spl character 7 not matured 8 stylish 9 simple pl reply
What's up? Do you want me to come online? If you are free we can talk sometimeï¿½
what be up do you want I to come online if you be free we can talk sometime
So u workin overtime nigpun?
so u workin overtime nigpun
Also sir, i sent you an email about how to log into the usc payment portal. I.ll send you another message that should explain how things are back home. Have a great weekend.
also sir I send you an email about how to log into the usc payment portal i.ll send you another message that should explain how thing be back home have a great weekend
Please Stay At Home. To encourage the notion of staying at home. All tax-paying citizens are entitled to ï¿½305.96 or more emergency refund. smsg.io/fCVbD
please stay at home to encou

In [42]:
i = 0
for x in df_teste["PROCESSED_TEXT"].duplicated():
    if x == True:
        print(i, [df_teste["PROCESSED_TEXT"].iloc[i]])
    i += 1

321 ['do you want a new video handset 750 any time any network min unlimited text camcorder reply or call now 08000930705 for del Sat am']
432 ['please call 08712402779 immediately as there be an urgent message wait for you']
483 ['XXXMobileMovieClub to use your credit click the wap link in the next txt message or click here > > http://wap xxxmobilemovieclub.com?n=qjkgighjjgcbl']
870 ['09066362231 URGENT your mobile no 07xxxxxxxxx win a â£2,000 bonus caller prize on 02/06/03 this be the 2nd attempt to reach you call 09066362231 asap']
929 ['free top ringtone -sub to weekly ringtone get 1st week free send subpoly to 81618-?3 per week stop sms-08718727870']
991 ['free MESSAGE activate your 500 free text message by reply to this message with the word free for term condition visit www.07781482378.com']
1035 ['the current lead bid be 151 to pause this auction send out Customer care 08718726270']
1067 ['449050000301 you have win a â£2,000 price to claim call 09050000301']
1213 ['free entry i

In [43]:
df_teste.drop([321, 483, 1035, 1213, 67, 1118], inplace=True)
df_teste.reset_index(drop=True, inplace= True)

In [44]:
df_teste.head(20)

,LABEL,TEXT,PROCESSED_TEXT
0,-1,Your opinion about me? 1. Over 2. Jada 3. Kusr...,your opinion about I 1 over 2 Jada 3 Kusruthi ...
1,-1,What's up? Do you want me to come online? If y...,what be up do you want I to come online if you...
2,-1,So u workin overtime nigpun?,so u workin overtime nigpun
3,-1,"Also sir, i sent you an email about how to log...",also sir I send you an email about how to log ...
4,1,Please Stay At Home. To encourage the notion o...,please stay at home to encourage the notion of...
5,1,BankOfAmerica Alert 137943. Please follow http...,BankOfAmerica alert 137943 please follow http:...
6,-1,Sorry dude. Dont know how i forgot. Even after...,sorry dude do not know how I forget even after...
7,-1,I don't quite know what to do. I still can't g...,I do not quite know what to do I still can not...
8,-1,Ok lor. Anyway i thk we cant get tickets now c...,ok lor anyway I thk we can not get ticket now ...
9,-1,Wat r u doing now?,wat r u do now


In [45]:
# TODO: Melhorar alguns lemmas, colocar stemm, retirar os caracteres especiais
display(df_teste["PROCESSED_TEXT"].iloc[1])
display(df_teste["TEXT"].iloc[1])

'what be up do you want I to come online if you be free we can talk sometime'

"What's up? Do you want me to come online? If you are free we can talk sometimeï¿½"

In [46]:
x = df_teste["PROCESSED_TEXT"]
y = df_teste["LABEL"]

In [47]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=42)

In [48]:
vectorizer = TfidfVectorizer()
x_train_tfidf = vectorizer.fit_transform(x_train)
x_test_tfidf = vectorizer.transform(x_test)

In [49]:
model = LogisticRegression(solver="lbfgs", multi_class="multinomial", max_iter=1000)
model.fit(x_train_tfidf, y_train)

LogisticRegression(max_iter=1000, multi_class='multinomial')

In [50]:
y_pred = model.predict(x_test_tfidf)

In [51]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do modelo: {accuracy:.2f}")

Acurácia do modelo: 0.91
